 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [8]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
import scipy.sparse as sp

##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number



Pandas version: 1.4.2
Numpy version: 1.21.5


In [14]:
%pwd #Return the current working directory path

'/home/emg/Documents/myPythonWay/github/epic'

In [15]:
!ls #List directory contents of files and directories

epic.ipynb  README.md


In [2]:
#Global variables
verbose_mode = False # verbose mode (True/False)
debug_mode = True # debug mode (True/False)
random_mode = True # random data generation  (True/False)
simulation_mode = True # simulation (True/False)

#Random_mode:
Generate aleatories PID numbers and symbols.



In [3]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [4]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

<class 'list'>


In [5]:
#TODO: Remove <1100 and > 9999
# send this tuples to a list for post revision

Series and DataFrames

In [6]:
pid_series = pd.Series(random_PID_symbols, random_PID_numbers)
pid_series = pid_series.sort_index(axis=0,ascending=True)
pid_series

1100     PSA-
1110        P
1116     QSA+
1137     PSA-
1141     SIA±
        ...  
9944    TICA±
9973    FICA+
9975        Y
9977    TICA±
9979        D
Length: 1000, dtype: object